# Spin Qubit Pulse Sequences with the HDAWG

This notebook shows you how to use pulse sequences with an HDAWG to realize various experiments. While the sequences are mostly general, they are adapted to those typically used for spin qubits by adding the gate pulsing to control between Coulomb and spin blockade.

# General Imports

In [2]:
%config IPCompleter.greedy=True

# import matplotlib.pyplot as plt
import numpy as np
import time
import ctypes
import helpers
import matplotlib.pyplot as plt
import gc
from picosdk.ps5000a import ps5000a as ps
from picosdk.functions import adc2mV, assert_pico_ok, mV2adc
import picoscope_module as pm
from scipy.fft import fft
import scipy.signal as signal
#install_token("LabOneQ.AllQuantumInOne.")
DO_EMULATION = False # run in emulation mode by default

timebase = 628 # 628
preTriggerSamples = 10
pico_sampling_rate = (timebase - 3) / 62500000
postTriggerSamples = int(np.ceil(45e-3/pico_sampling_rate)) #Measurment length was 8
TriggerLevel=1
for rep in range(1):
    print(rep)
    maxSamples = preTriggerSamples + postTriggerSamples
    ready = ctypes.c_int16(0)
    check = ctypes.c_int16(0)
    maxADC = ctypes.c_int16()



    chandle, status = pm.initialize_picoscope()
    chARange, chBRange, status = pm.setup_channels(chandle)
    # find maximum ADC count value

    pm.setup_trigger_noTrigger(chandle, chARange, chBRange, maxADC,TriggerLevel)

    # Get timebase information

    # define experiment
    NUM_REP = 30 # WHY DOES 8 work all the time?

    source = ps.PS5000A_CHANNEL["PS5000A_CHANNEL_B"] #set the buffer
    noOfCaptures = NUM_REP
    cmaxSamples = ctypes.c_int32(maxSamples)

    # Create time data
    time_stamp=int(time.time())

    with open('Data/data_%s.npy'%(time_stamp), 'wb') as f:
                status,timeIntervalns = pm.capture_rapid_data_block(chandle, preTriggerSamples, postTriggerSamples, timebase, maxSamples,noOfCaptures)
                while ready.value == check.value:
                    status["isReady"] = ps.ps5000aIsReady(chandle, ctypes.byref(ready))
                print('data acq')
                buffersMax, buffersMin = pm.create_rapid_buffer(chandle, source, maxSamples, noOfCaptures)
                pm.getValuesRapid(chandle,maxSamples,noOfCaptures)
                print('done reading')

                time_array = np.linspace(0, (cmaxSamples.value - 1) * timeIntervalns.value, cmaxSamples.value)
                np.save(f,time_array)
                print('saved time')
                for i in range(noOfCaptures):
                    np.save(f,adc2mV(buffersMax[i], chARange, maxADC))


    Vdet_list=np.array([])
    for i in range(noOfCaptures):
            Vdet_list=np.append(Vdet_list,adc2mV(buffersMax[i], chARange, maxADC))



        



    # Stop the scope
    status["stop"] = ps.ps5000aStop(chandle)
    assert_pico_ok(status["stop"])


    # # Close unit Disconnect the scope
    status["close"]=ps.ps5000aCloseUnit(chandle)
    assert_pico_ok(status["close"])

    fig4, ax4=plt.subplots()
    ax4.plot(np.linspace(0, 1e5-1,len(Vdet_list)),  np.abs(fft(Vdet_list)) , alpha=0.9, label='with RF lines')
    ax4.set_title("FFT_Zoom")
    ax4.set_xlim(0, 1e3)
    ax4.set_yscale('log')
    ylim=ax4.get_ylim()
    xlim=ax4.get_xlim()
    ax4.set_xlabel("Frequency (Hz)")
    ax4.set_ylabel("FFT signal (a.u.)")
    peaks=signal.find_peaks(np.abs(fft(Vdet_list)), height=1e3, distance=100)
    peak_freq=np.linspace(0, 1e5-1,len(Vdet_list))[peaks[0]]
    ax4.legend()
    for i in range(0, len(peak_freq)):
        if peak_freq[i]<max(xlim):
            ax4.vlines(peak_freq[i], *ylim, color='red', alpha=0.3)
            ax4.text(peak_freq[i], max(ylim), '%.0lf'%peak_freq[i], fontsize='small')
            print("Frequenz %.2lf hat eine Höhe von %.2e" %(peak_freq[i], peaks[1]['peak_heights'][i]))

    fig, ax= plt.subplots()
    ax.plot(Vdet_list)

0
data acq
done reading
saved time


: 

: 